In [1]:
import os
import sys

import pandas as pd
import stanza
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import nltk
import plotly.express as px
import string

/Users/svengerloff/Library/Mobile Documents/com~apple~CloudDocs/Documents/Uni/TU_WIEN/WS2024_25/NLP/online_sexism_detection/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/svengerloff/Library/Mobile Documents/com~apple~CloudDocs/Documents/Uni/TU_WIEN/WS2024_25/NLP/online_sexism_detection/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append('..')

from utils.preprocessing import dummy_function
dummy_function('')

Hello World


In [3]:
os.listdir(file_path)

NameError: name 'file_path' is not defined

In [4]:
file_path = os.path.join(os.getcwd(), '..', 'data')
df = pd.read_csv(file_path + '\edos_labelled_aggregated.csv')
df = df[["text","label_sexist","split"]]
df.head(5)

,rewire_id,text,annotator,label_sexist,label_category,label_vector,split
0,sexism2022_english-0,[USER] I wonder what keeps that witch looking ...,17,sexist,2. derogation,2.2 aggressive and emotive attacks,train
1,sexism2022_english-0,[USER] I wonder what keeps that witch looking ...,2,sexist,2. derogation,2.2 aggressive and emotive attacks,train
2,sexism2022_english-0,[USER] I wonder what keeps that witch looking ...,6,not sexist,none,none,train
3,sexism2022_english-1,"What do you guys think about female ""incels""? ...",17,not sexist,none,none,train
4,sexism2022_english-1,"What do you guys think about female ""incels""? ...",15,not sexist,none,none,train


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   rewire_id       60000 non-null  object
 1   text            60000 non-null  object
 2   annotator       60000 non-null  int64 
 3   label_sexist    60000 non-null  object
 4   label_category  60000 non-null  object
 5   label_vector    60000 non-null  object
 6   split           60000 non-null  object
dtypes: int64(1), object(6)
memory usage: 3.2+ MB


In [6]:
df.rename(columns={'label_sexist': 'label'}, inplace=True)
df['label'] = df['label'].map({'sexist': 1, 'not sexist': 0})

In [10]:
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stanza.download('en')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/svengerloff/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/svengerloff/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-10-29 12:42:37 INFO: Downloaded file to /Users/svengerloff/stanza_resources/resources.json
2024-10-29 12:42:37 INFO: Downloading default packages for language: en (English) ...
2024-10-29 12:42:39 INFO: File exists: /Users/svengerloff/stanza_resources/en/default.zip
2024-10-29 12:42:44 INFO: Finished downloading models and saved to /Users/svengerloff/stanza_resources


In [11]:
nlp = stanza.Pipeline('en', processors='tokenize,lemma')

2024-10-29 12:42:46 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-29 12:42:46 INFO: Downloaded file to /Users/svengerloff/stanza_resources/resources.json
2024-10-29 12:42:46 WARNING: Language en package default expects mwt, which has been added
2024-10-29 12:42:46 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| lemma     | combined_nocharlm |

2024-10-29 12:42:46 INFO: Using device: cpu
2024-10-29 12:42:46 INFO: Loading: tokenize
/Users/svengerloff/Library/Mobile Documents/com~apple~CloudDocs/Documents/Uni/TU_WIEN/WS2024_25/NLP/online_sexism_detection/.venv/lib/python3.9/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=Fals

In [12]:
df_subset = df.head(100)

In [13]:
nlp = stanza.Pipeline('en', processors='tokenize,lemma,pos')

2024-10-29 12:42:56 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-29 12:42:56 INFO: Downloaded file to /Users/svengerloff/stanza_resources/resources.json
2024-10-29 12:42:56 WARNING: Language en package default expects mwt, which has been added
2024-10-29 12:42:57 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

2024-10-29 12:42:57 INFO: Using device: cpu
2024-10-29 12:42:57 INFO: Loading: tokenize
/Users/svengerloff/Library/Mobile Documents/com~apple~CloudDocs/Documents/Uni/TU_WIEN/WS2024_25/NLP/online_sexism_detection/.venv/lib/python3.9/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `t

In [49]:
# Function to clean text and remove specific strings
def clean_text(text):
    return text.replace('[USER]', '').strip()  # Remove '[USER]' and any leading/trailing whitespace

# Function to lemmatize text and return as a list of lemmas
def lemmatize_text_to_list(text):
    doc = nlp(text)
    sentences = [doc]
    lemmatized_words = [word.lemma for sentence in doc.sentences 
                        for word in sentence.words if word.lemma not in stopwords.words('english')]
    lemmatized_words = [word.lower() for word in lemmatized_words if len(word) >= 2 or word=="I"]
    return lemmatized_words  # Return as a list

In [15]:
import stanza
stanza.download('en')       # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in English
doc = nlp("Barack Obama was born in Hawaii.  He was elected president in 2008.")
doc.sentences[0].print_dependencies()

2024-10-29 12:43:35 INFO: Downloaded file to /Users/svengerloff/stanza_resources/resources.json
2024-10-29 12:43:35 INFO: Downloading default packages for language: en (English) ...
2024-10-29 12:43:37 INFO: File exists: /Users/svengerloff/stanza_resources/en/default.zip
2024-10-29 12:43:40 INFO: Finished downloading models and saved to /Users/svengerloff/stanza_resources
2024-10-29 12:43:40 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-29 12:43:40 INFO: Downloaded file to /Users/svengerloff/stanza_resources/resources.json
2024-10-29 12:43:42 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        

('Barack', 4, 'nsubj:pass')
('Obama', 1, 'flat')
('was', 4, 'aux:pass')
('born', 0, 'root')
('in', 6, 'case')
('Hawaii', 4, 'obl')
('.', 4, 'punct')


In [16]:

# Clean the 'text' column
df_subset['cleaned_text'] = df_subset['text'].apply(clean_text)

# Apply the lemmatization function to the cleaned 'text' column
df_subset['lemmatized_list'] = df_subset['cleaned_text'].apply(lemmatize_text_to_list)

# Display the updated DataFrame with the new columns
print(df_subset[['text', 'cleaned_text', 'lemmatized_list']].head())

NameError: name 'clean_text' is not defined

In [50]:
lemmatize_text_to_list("In Nigeria, if you rape a woman, the men rape you back! #NSFW")

['nigeria', 'rape', 'woman', 'man', 'rape', 'back', '#nsfw']

In [52]:
df_subset.head()

,text,label,split,cleaned_text,lemmatized_list
0,"In Nigeria, if you rape a woman, the men rape ...",0,dev,"In Nigeria, if you rape a woman, the men rape ...","[nigeria, rape, woman, man, rape, back, #nsfw,..."
1,"Then, she's a keeper. 😉",0,train,"Then, she's a keeper. 😉",[keeper]
2,This is like the Metallica video where the poo...,0,train,This is like the Metallica video where the poo...,"[like, metallica, video, poor, mutilate, basta..."
3,woman?,0,train,woman?,[woman]
4,I bet she wished she had a gun,0,dev,I bet she wished she had a gun,"[i, bet, wish, gun]"


In [ ]:
stanza.download('en')  # Ensure the model is downloaded if not already
nlp = stanza.Pipeline('en', processors='tokenize,lemma,pos')

# Select the first 100 rows
df_subset = df

# Function to clean text
def clean_text(text):
    return text.replace('[USER]', '').strip()

# Function to lemmatize text and return as a list of lemmas
def lemmatize_text_to_list(text):
    doc = nlp(text)
    lemmatized_words = [word.lemma for sentence in doc.sentences for word in sentence.words]
    return ' '.join(lemmatized_words)  # Join lemmas into a single string

# Clean and lemmatize the 'text' column
df_subset['cleaned_text'] = df_subset['text'].apply(clean_text)
df_subset['lemmatized_text'] = df_subset['cleaned_text'].apply(lemmatize_text_to_list)


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will 

In [ ]:
import plotly.express as px
# Step 1: Vectorize the text data using TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_subset['lemmatized_text']).toarray()  # Convert sparse matrix to dense

# Step 2: Apply PCA to reduce dimensionality to 2D
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X)

# Step 3: Create a DataFrame for PCA results
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['label'] = df_subset['label'].values  # Add labels to the DataFrame
pca_df['index'] = pca_df.index  # Add index for hover information

# Step 4: Plot the PCA results using Plotly
fig = px.scatter(
    pca_df,
    x='PC1',
    y='PC2',
    color='label',
    hover_data={'index': True, 'label': True},  # Show index and label on hover
    title='PCA of TF-IDF Vectors',
    color_discrete_sequence=px.colors.qualitative.Vivid  # Optional color palette
)

# Update layout
fig.update_layout(
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    legend_title='Label',
)

# Show the plot
fig.show()

## Data exploration and experimental analysis

We start with finding the most frequently used words overall

In [112]:
from collections import Counter

frequency_words = Counter()
for word_list in df_subset['lemmatized_list']:
    frequency_words.update(word_list)
    
print(frequency_words.most_common(n=15))

[('i', 56), ('woman', 27), ('get', 17), ('man', 14), ('url', 13), ('say', 13), ('like', 11), ('go', 11), ('would', 10), ('make', 10), ('rape', 9), ('want', 9), ('fuck', 9), ("'s", 8), ('shit', 8)]


We can compare this to the class-wise numbers

In [126]:
from collections import Counter

n=15

frequency_words_sexist = Counter()
frequency_words_nonsexist = Counter()
for row in df_subset.itertuples(index=False):
    word_list = row[4]
    if row[1]==1:
        frequency_words_sexist.update(word_list)
    else:
        frequency_words_nonsexist.update(word_list)

print(frequency_words_sexist.most_common(n=n))


[('i', 21), ('woman', 12), ('get', 8), ('go', 8), ('bitch', 8), ('man', 7), ('make', 5), ('hot', 4), ('pussy', 4), ('want', 4), ('time', 4), ('say', 4), ('fuck', 4), ("'s", 4), ('would', 4)]


In [127]:
print(frequency_words_nonsexist.most_common(n=n))

[('i', 35), ('woman', 15), ('url', 13), ('say', 9), ('get', 9), ('rape', 8), ('like', 8), ('man', 7), ('would', 6), ('one', 5), ('make', 5), ('fuck', 5), ('person', 5), ('want', 5), ('see', 5)]


In [183]:
df_commonwords = pd.DataFrame(frequency_words_nonsexist.most_common(n=n)).rename(columns={0: "Word", 1: "Nonsexist"})
df_commonwords_2 = pd.DataFrame(frequency_words_sexist.most_common(n=15)).rename(columns={0: "Word", 1: "Sexist"})
df_commonwords = (df_commonwords.set_index("Word")
                                  .join(df_commonwords_2
                                        .set_index("Word"), how="outer")
                                  .reset_index()
                                  .fillna(0))
df_commonwords.loc[:, "Sum"] = df_commonwords["Sexist"] + df_commonwords['Nonsexist']

df_commonwords = (df_commonwords.melt(id_vars=["Word", "Sum"], value_vars=["Sexist", "Nonsexist"], value_name="Frequency")
                  .sort_values(by="Sum", axis=0, ascending=False).reset_index(drop=True)).rename(columns={"variable": "Class"})

In [184]:
df_commonwords

,Word,Sum,Class,Frequency
0,i,56.0,Sexist,21.0
1,i,56.0,Nonsexist,35.0
2,woman,27.0,Nonsexist,15.0
3,woman,27.0,Sexist,12.0
4,get,17.0,Sexist,8.0
5,get,17.0,Nonsexist,9.0
6,man,14.0,Nonsexist,7.0
7,man,14.0,Sexist,7.0
8,url,13.0,Sexist,0.0
9,say,13.0,Sexist,4.0


In [185]:
fig = px.bar(df_commonwords, x="Word", y="Frequency", color="Class")
fig.show()

## Post Length analysis

In [203]:
df_subset.loc[:,'post_length'] = df_subset['lemmatized_list'].apply(len)
fig = px.violin(df_subset, y='post_length', color='label', points='outliers', box=True,
                labels={'post_length': 'Number of Words (processed)',
                        'label': 'Classified as Sexist (N/Y),'},
                title="Distribution of post length per class")
fig.update_layout(autosize=False, width=800, height=500)
fig.show()

## Class distribution analysis

In [206]:
fig = px.histogram(df_subset, color='label', x='split', barmode='group', barnorm='percent')
fig.update_layout(title_text="Distribution of classes in all splits", xaxis_title="Set Split", yaxis_title="Class frequency (%)")
fig.update_layout(bargap=0.1, autosize=False, width=800, height=500)
fig.show()

## Sentiment Analysis

In [187]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...


In [190]:
sid = SentimentIntensityAnalyzer()

def get_sentiment_vader(post):
    sentiment_scores = sid.polarity_scores(post)
    return sentiment_scores['compound']  # Compound score is an overall sentiment score

# Apply sentiment function to each post
df_subset.loc[:, 'sentiment_score'] = df_subset['text'].apply(get_sentiment_vader)

# Define sentiment category based on compound score
df_subset.loc[:, 'sentiment'] = df_subset['sentiment_score'].apply(
    lambda score: 'positive' if score > 0 else 'negative' if score < 0 else 'neutral'
)

In [207]:
fig = px.violin(df_subset, y='sentiment_score', color='label', box=True, points='outliers',
                        labels={'post_length': 'Sentiment Score',
                                     'label': 'Classified as Sexist (N/Y),'},
                        title="Distribution of sentiment Scores per class")
fig.update_layout(autosize=False, width=800, height=500)
fig.update_layout(xaxis_title="Nonsexist/Sexist", yaxis_title="Sentiment Score")
fig.show()